# Calculation of Self Solvation Free Energy

## Do Imports

In [ ]:
import os
from rdkit import Chem
import pygromos
from pygromos.files.forcefield.openff.openff import OpenFF
from pygromos.files.gromos_system.gromos_system import Gromos_System
from pygromos.simulations.approaches.solvation_free_energy_calculation.solvation_free_energy import Solvation_free_energy_calculation
from pygromos.simulations.hpc_queuing.submission_systems.local import LOCAL

## Choose Molecule to run calculation for

In [ ]:
smiles = "c1ccccc1"
workfolder = project_dir = os.path.abspath(os.path.dirname(pygromos.__file__)+"/../examples/example_files/solvation_free_energy")

### create the gromos_system from a smile and get the number of atoms

In [ ]:
groSys = Gromos_System(work_folder=workfolder, system_name="test", in_smiles=smiles, forcefield=OpenFF(), auto_convert=True)
number_of_atoms = groSys.mol.GetNumAtoms()
print("Number of atoms:", number_of_atoms)

In [ ]:
subSys = LOCAL() # use the local submission system (for cluster use LSF)

In [ ]:
n_points = 5 # Number of Lambda points to calculate (typically 21)

## Create The Solvation free energy calculation system

In [ ]:
sf = Solvation_free_energy_calculation(input_system=groSys, # Gromos_System, SMILES (str) or rdkit Mol
                                       work_folder=workfolder, # Folder to do calculations in
                                       system_name="test", # Name of the system (does not need to be smiles but convenient)
                                       forcefield=OpenFF(), # Force field to use
                                       density=789, # density of the liquid in kg/L
                                       num_molecules=512, # number of molecules used for the calculation
                                       num_atoms=number_of_atoms, # number of atoms in one molecule
                                       subsystem=subSys, # Subsystem to use for calculation local or lsf
                                       amberscaling=False, # Whether to use amberscaling (for openforcefield recommended)
                                       n_points=n_points) # Number of Lambda points to calculate (typically 21)

### Create Liquid

In [ ]:
sf.create_liq()

### Minimize Liquid

In [ ]:
emin_sys, jobID = sf.minimize_liq(in_gromos_simulation_system=sf.groSys_liq,prev_JobID=-1)

### Change the number of cores for longer runs

In [ ]:
sf.subsystem.nomp = 6

## Equilibrate System

In [ ]:
eq_sys, jobID = sf.eq_liq(in_gromos_simulation_system=emin_sys,prev_JobID=jobID)

## Do TI calculation

In [ ]:
ti_sys, jobID = sf.ti_liq(in_gromos_simulation_system=eq_sys,
                          prev_JobID=jobID,
                          n_points=n_points)

## Read out results

In [ ]:
sf.calculate_solvation_free_energy(n_points=n_points)